In [ ]:
from verifier_utility import create_jsonl_COT, create_jsonl_NCOT
from gpt_utility import df,client,save_file_response_as_jsonl,read_jsonl_file, get_response_label,read_csv
#create batches
model_name_cot = '[Your model ID for COT]'
model_name_ncot = '[Your model ID for NCOT]'
create_jsonl_NCOT('output/pii_ft_detected_2.csv', df,model_name_ncot)
create_jsonl_COT('output/pii_ft_detected_2.csv', df, model_name_cot)



In [ ]:
ncot_file_path = 'output/ft+ft_testing_ncot.jsonl'
cot_file_path = 'output/ft+ft_testing_cot.jsonl'
#This and next cell submit batch for ncot
batch_input_file = client.files.create(
  file=open(ncot_file_path, "rb"),
  purpose="batch"
)
batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

In [ ]:
client.batches.retrieve("[file_id here]")

In [ ]:
#This and next cell submit batch for cot
batch_input_file = client.files.create(
  file=open(cot_file_path, "rb"),
  purpose="batch"
)
batch_input_file_id = batch_input_file.id
client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "nightly eval job"
    }
)

In [ ]:
client.batches.retrieve("[file_id here]")

In [ ]:
#once the above two inferenece batch is completed, retreive them 
file_response_ncot = client.files.content('[id here]')
file_response_cot = client.files.content('[id here]')
save_file_response_as_jsonl(file_response_ncot, 'output/file_resp_ncot.jsonl')
save_file_response_as_jsonl(file_response_cot, 'output/file_resp_cot.jsonl')
json_lst_ncot = read_jsonl_file(file_response_ncot)
json_lst_cot = read_jsonl_file(file_response_ncot)
pii_ft_detected_2 = read_csv('output/pii_ft_detected_2.csv')


def filter_detected_entities():
    for json_lst in [json_lst_ncot, json_lst_cot]:
        res_lst = []
        for item in json_lst: 
            content = item['response']['body']['choices'][0]['message']['content']
            label = get_response_label(content)
            res_lst.append(label)
        mask = [x == 'T' for x in res_lst]
        filtered_df = pii_ft_detected_2[mask]

        if json_lst == json_lst_ncot:
            filtered_df.to_csv(f'output/pii_ft_detected_ncot.csv', index=False)
        elif json_lst == json_lst_cot:
            filtered_df.to_csv(f'output/pii_ft_detected_cot1.csv', index=False)
filter_detected_entities()
#save the result in .csv format.
